# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
import time, numpy as np, os, re

# homemade stuff
import gimbal_control as gc
from extended_kalman_filter import KalmanFilter, ExtendedKalmanFilter
from image_classifier import ImageClassifier
from utils import plot_util, ExperimentLogger, GimbalAngleBuffer

# Constants + things which need global scope

In [ ]:
total_run_time = 30   # seconds
t_loop_desired = 0.05 # seconds = 50 ms

# initialize variables in case the neural net doesn't spot anything the first time round
obj_yaw, obj_pitch = 0, 0

## A Basic Loop

In [ ]:
# somewhat high R to avoid some jerkiness
EKF_yaw   = KalmanFilter(Ts=0.05, Q=1, R=1, a=0.0)

# very high R to make the thing respond less to my bottom half disappearing behind a chair
EKF_pitch = KalmanFilter(Ts=0.05, Q=0.5, R=50, a=0.0)

IC = ImageClassifier(
    graph_filename='../Models/MobileNet_SSD_caffe/graph',
    label_filename='../Models/MobileNet_SSD_caffe/categories.txt',
    class_of_interest='person',
    camera_resolution=(1640, 1232),#(1640, 922)
    camera_FOV_deg=(62.2, 48.8),
    photo_logging_params=(5, 'logged_photos/'),
    debug=False)

# logging used for offline stuff only
EL = ExperimentLogger()

# a rolling buffer for the gimbal angles
gimbal_buffer = GimbalAngleBuffer()

# sometimes the gimbal adds 360ᵒ or more to the angle. This keeps it to [-180, 180]
def remove_spikes(x, limit=180): return ((x + limit) % (2*limit)) - limit

In [ ]:
while IC.dict_queue.empty() is True: pass # wait until there's a result from the nn (even if it didn't spot an object)

t_start = time.time()
t_since_nn_update = time.time()

EKF_yaw.start_timer()
EKF_pitch.start_timer()

while True:
    t = time.time()
    
    ######################### LOG THE GIMBAL ANGLES #########################
    gc_angles = gc.get_motor_angles()
    gc_angles['yaw'] = remove_spikes(gc_angles['yaw']) # get rid of erroneous spikes
    gc_angles['pitch'] = remove_spikes(gc_angles['pitch'])
    gimbal_buffer.log(gc_angles)
    
    ######################### NN and EKF #########################
    # if there is a new result waiting...
    if not IC.dict_queue.empty():
        bb, bb_angles, photo_time = IC.get_result()

        # and there was an actual object detected...
        if bb_angles != -1:
            (obj_x1, obj_y1), (obj_x2, obj_y2) = bb_angles
            obj_yaw = (obj_x1 + obj_x2)/2
            obj_pitch = -(obj_y1 + obj_y2)/2 # come as negative angles
            
#             print('photo -> result =', (time.time()-photo_time)*1000)
            logger_yaw, logger_pitch = gimbal_buffer.angle_closest_to(photo_time)            
            EKF_yaw.better_update(obj_yaw + logger_yaw)
            EKF_pitch.better_update(obj_pitch + logger_pitch)
    
    ######################### CONTROL #########################
    desired_yaw = EKF_yaw.get_cur_est_pos()
    desired_pitch = EKF_pitch.get_cur_est_pos()
    
    # add in bounds
    desired_pitch = min(max(desired_pitch, -45), 45) # bounds are (-45, 45)
    desired_yaw = min(max(desired_yaw, -120), 120)   # bounds are (-120, 120)
    
    gc.send_angle_command(roll=0, pitch=desired_pitch, yaw=desired_yaw)

    ######################### TIMING #########################    
    t_end = time.time()
    if t_end > t_start + total_run_time:
        break
    elif t_end - t > t_loop_desired:
        EL.log(gc_angles, obj_yaw, EKF_yaw, obj_pitch, EKF_pitch, time.time() - t_start)
    else:
        time.sleep(t_loop_desired - (t_end - t))  # aim for a loop time of 50ms
        EL.log(gc_angles, obj_yaw, EKF_yaw, obj_pitch, EKF_pitch, time.time() - t_start)

IC.close()
gc.send_angle_command(0, 0, 0);

In [ ]:
# IC.close()
# gc.send_angle_command(0, 0, 0);

In [ ]:
# very hacky! should make this nicer!

potential_dirs = [f for f in os.listdir('.') if re.match(r'logged_photos*', f)]
# print(potential_dirs)
if len(potential_dirs) == 1:
    EL_save_dir = potential_dirs[0] + '/ExperimentLogger_readings.csv'
else:
    EL_save_dir = potential_dirs[-2] + '/ExperimentLogger_readings.csv' # -1 = logged_photos, others are sorted

print('Saving experiment logger to ' + EL_save_dir)
EL.save(EL_save_dir)

In [ ]:
# EL.plot()